In [48]:
import numpy as np
import pandas as pd
import gensim.downloader as api

from scipy.spatial import distance
from scipy.spatial.distance import cosine
from scipy import sparse

import csv
import sys

In [6]:
model = api.load("word2vec-ruscorpora-300")

In [70]:
model.get_vector('сахар_NOUN')
# model.most_similar(positive=[''], negative=[''])

numpy.ndarray

In [7]:
data = pd.read_csv('data_for_w2v.csv')

In [8]:
def to_vector(words):                                                   # функция для создания вектора отдельного курса
    request_vect = [i * 0 for i in range(300)]
    for i in words:
        try: 
            vect_word = model.get_vector(i)
            for j in range(300):
                request_vect[j] += vect_word[j]
        except KeyError: pass
    return np.asarray(request_vect)

In [9]:
dt = []
for i in range(len(data)):
    dt.append(to_vector(data.all_info[i].split()))

In [10]:
lst = pd.DataFrame([i] for i in range(10000))

In [11]:
vectorize = pd.DataFrame(columns=['vector'])

In [12]:
for i in dt:
    vectorize = vectorize.append({'vector': i}, ignore_index=True)

In [13]:
vectorize = data.join(vectorize).drop('all_info', 1)

In [14]:
vectorize = vectorize.rename(columns={'Unnamed: 0': 'X', '0': 'Y'}, inplace=False)

In [15]:
vectorize = vectorize.drop('X', 1)

In [23]:
sort_dict = lambda dictn: {key: value for key, value in sorted(dictn.items(), key=lambda item: item[1])}

In [45]:
def recommendation_system_with_cosine(request):
    simularity_course = {}
    for i in vectorize.content_id:
        simularity_course[i] = cosine(vectorize.vector[vectorize.content_id == request].values[0],
                                        vectorize.vector[vectorize.content_id == i].values[0])
    simularity_course = sort_dict(simularity_course)
    return simularity_course.keys()
    

In [21]:
def recommendation_system_with_euclidean(request):
    simularity_course = {}
    for i in vectorize.content_id:
        simularity_course[i] = distance.euclidean(vectorize.vector[vectorize.content_id == request].values[0],
                                        vectorize.vector[vectorize.content_id == i].values[0])
    simularity_course = sort_dict(simularity_course)
    return simularity_course.keys()
    

In [68]:
courses = pd.read_csv('contents.csv', sep=',', encoding='utf16')

In [77]:
vectorize

,content_id,vector
0,000249F5-15E7-40FA-A835-CB6D34EF355E,"[2.1684414296760224, -1.7592279258242343, -3.0..."
1,0008020A-E57E-40BD-9196-AA20637936E1,"[0.5116495145484805, -0.12669147865381092, -0...."
2,000DDF9E-81D8-4612-A4DF-07E70705E0C9,"[6.7457976980367675, 1.9754356873454526, -3.48..."
3,000E9424-369E-43DD-BC66-D46A6E7A995C,"[1.2473489625554066, -2.4715613400767324, -2.0..."
4,00114246-96FC-4155-A563-EA55C52A5596,"[1.2040695578616578, -0.3138086749240756, -2.5..."
...,...,...
9995,54AC2EDC-661C-4A5C-AF64-CEE9ECE5BFC0,"[4.046894671861082, -3.7551842789398506, -2.34..."
9996,54AC3B6A-B59C-47B3-9F2A-EB3EBE7F5A4D,"[1.2875328328227624, -1.8242090779385762, -8.0..."
9997,54ACE266-6B6F-4F05-8466-DCF0E904D66E,"[1.0324313035525847, -0.7359280334785581, -0.9..."
9998,54AE820A-A78B-475D-AD86-E8B41CE18EE1,"[3.2853139113867655, -0.5581432358594611, -7.9..."


In [115]:
courses.sample(5)

,content_id,content_type_id,rpd_discipline_id,enable_type_id,enable_type_change_date,normal_name,edition_number,bdescription,ap_datefinish,Bdate,preview_bnumber,real_bnumber,new_edition_exists,bbuyers_count,is_mediacourse,fits_exam
129,00EDF109-109C-42D9-98E1-65CF828D6D3C,1,4352,4,2020-04-01 18:01:25,Биология,2.0,В учебнике значительное место уделено описанию...,2032-06-01 00:00:00,2010-09-30 00:00:00,45.0,454.0,1,211,0,0
10890,447D9452-73F9-40D3-883C-6A2CAEB50005,1,15703,4,2020-04-01 18:01:25,Основы виноделия,2.0,В книге использованы новые и проверенные време...,2032-09-22 00:00:00,2018-03-30 00:00:00,42.0,423.0,1,5,0,0
29918,B9AAF0F1-94CE-4DB7-AA2C-29DFDF121A8A,1,7912,2,2019-07-13 02:21:58,История государства и права России,1.0,Непосредственной сдаче экзамена или зачета по ...,2102-08-26 00:00:00,2008-08-08 00:00:00,22.0,222.0,1,70,0,0
24925,9A94CA51-F00A-4864-B2CB-1DD8E8A8B5A7,1,0,2,2019-07-13 02:21:58,Уголовное право. Особенная часть. Практикум,1.0,Практикум представляет собой пособие для прове...,2029-09-22 00:00:00,2015-01-30 00:00:00,45.0,450.0,1,29,0,0
39939,F8A1E03D-52DA-40C7-89ED-656AF0C31B31,1,11612,1,2020-03-19 05:05:47,Прогнозирование ползучести и длительной прочно...,2.0,Изложены основные сведения по ползучести и дли...,NaN,2018-06-18 00:00:00,13.0,136.0,0,8,0,0


In [120]:
for i in list(recommendation_system_with_cosine('00EDF109-109C-42D9-98E1-65CF828D6D3C'))[1:11]: 
    print(*list(courses.normal_name[courses.content_id == i]))

Биология
Биология
Биология
Биология
Биология
Биология. Углубленный курс
Биология в 2 т. Том 1
Генетика
Генетика. В 2 ч. Часть 1
Генетика. В 2 ч. Часть 1


In [121]:
for i in list(recommendation_system_with_euclidean('00EDF109-109C-42D9-98E1-65CF828D6D3C'))[1:11]: 
    print(*list(courses.normal_name[courses.content_id == i]))

Биология
Биология
Биология
Биология
Биология
Микробиология: теория и практика в 2 ч. Часть 1
Микробиология: теория и практика в 2 ч. Часть 1
Биохимия человека
Концепции современного естествознания
Концепции современного естествознания
